# What is `uarray`?
At its core, `uarray` is a dispatch and back-end mechanism specifically geared towards array computing. Combined with its sister packages `unumpy` (and others currently in development), it allows NumPy functions to be overridden by their counterparts in other libraries (such as Dask, Xnd, and so on) while using the exact same code everywhere. Backends can be changed using just a context manager.

Please note that only a small subset of the NumPy API is implemented, and not every backend implements every API method.

In [1]:
import uarray as ua
import unumpy as np  # Note the changed import statement

In [2]:
import unumpy.xnd_backend as XndBackend
import unumpy.numpy_backend as NumpyBackend
import unumpy.torch_backend as TorchBackend

ua.set_global_backend(NumpyBackend)
ua.register_backend(XndBackend)
ua.register_backend(TorchBackend)

## Computing on different back-ends
`unumpy` allows you to compute with different back-ends. Here are examples of creating arrays via `unumpy` (something not currently possible with NEP-18, the `__array_function__` protocol).

In [3]:
my_list = [0, 1, 2, 3, 4]

with ua.set_backend(XndBackend):
    x = np.array(my_list)
print('With XndBackend set, the type of the array is: {}'.format(type(x)))

with ua.set_backend(TorchBackend):
    y = np.array(my_list)
print('With TorchBackend set, the type of the array is: {}'.format(type(y)))

with ua.set_backend(NumpyBackend):
    z = np.array(my_list)
print('With NumpyBackend set, the type of the array is: {}'.format(type(z)))

With XndBackend set, the type of the array is: <class 'xnd.array'>
With TorchBackend set, the type of the array is: <class 'torch.Tensor'>
With NumpyBackend set, the type of the array is: <class 'numpy.ndarray'>


## Computing based on the type of array passed in
`unumpy` allows you to compute on arrays based on the type, in a fashion similar to NEP-18.

In [4]:
print('With np.sum(xnd_array) we get: {}'.format(type(np.sum(x))))
print('With np.sum(torch_array) we get: {}'.format(type(np.sum(y))))
print('With np.sum(numpy_array) we get: {}'.format(type(np.sum(z))))

With np.sum(xnd_array) we get: <class 'xnd.array'>


TypeError: sum() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, ), but expected one of:
 * (torch.dtype dtype)
 * (tuple of names dim, bool keepdim, torch.dtype dtype)
 * (tuple of ints dim, bool keepdim, torch.dtype dtype)


## Forcing a particular backend
You can even force a particular back-end, if you want to pipe all possible computations through that back-end.

In [5]:
with ua.set_backend(TorchBackend, coerce=True):
    print('Using the TorchBackend with coerce=True on a NumPy array: {}'.format(type(np.sum(z))))

Using the TorchBackend with coerce=True on a NumPy array: <class 'torch.Tensor'>
